<a href="https://colab.research.google.com/github/TAlkam/Data-Driven-Subtyping-of-Alzheimer-s-Emergency-Presentations-Using-Unsupervised-Machine-Learning/blob/main/Data_Driven_Subtyping_of_Alzheimer%E2%80%99s_Emergency_Presentations_Using_Unsupervised_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install pandas pyreadstat openpyxl --quiet

import pandas as pd

# === Load NEDS 2022 data (Stata .dta format) ===
neds_path = 'NEDS_2022_Core Alzheimers only.dta'
neds = pd.read_stata(neds_path, convert_categoricals=False)

# === Load ICD-10 Code Label Mapping ===
map_path = 'section111validicd10-jan2025_0.xlsx'
icd_map = pd.read_excel(map_path)

# Preview mapping structure
icd_map.columns  # should include 'icd10cm_code' and 'label' or similar

# Clean up ICD map
# Clean column names
icd_map.columns = icd_map.columns.str.strip()

# Rename for clarity
icd_map = icd_map.rename(columns={
    'CODE': 'code',
    'LONG DESCRIPTION (VALID ICD-10 FY2025)': 'label'
})

# Standardize formatting
icd_map['code'] = icd_map['code'].str.strip().str.upper()
icd_map['label'] = icd_map['label'].str.strip()

icd_map['code'] = icd_map['code'].str.strip().str.upper()

# === Flatten diagnosis columns ===
diag_cols = [col for col in neds.columns if col.startswith('i10_dx')]
neds_diag = neds[['age', 'key_ed'] + diag_cols].copy()

# Melt to long format
neds_long = neds_diag.melt(id_vars=['key_ed', 'age'], value_vars=diag_cols,
                           var_name='dx_index', value_name='icd10_code')

# Drop missing codes
neds_long = neds_long.dropna(subset=['icd10_code'])
neds_long['icd10_code'] = neds_long['icd10_code'].str.strip().str.upper()

# === Merge with ICD-10 labels ===
neds_labeled = neds_long.merge(icd_map, how='left', left_on='icd10_code', right_on='code')

# Handle unmapped codes if needed
unmapped = neds_labeled[neds_labeled['label'].isna()]['icd10_code'].unique()
print(f"⚠️ Unmapped ICD-10 codes: {len(unmapped)}")

# Drop null labels or fill with code if desired
neds_labeled = neds_labeled.dropna(subset=['label'])

# === Create binary matrix: rows = visit, cols = comorbidity label ===
binary_matrix = pd.crosstab(neds_labeled['key_ed'], neds_labeled['label'])
binary_matrix = binary_matrix.clip(upper=1)  # Ensure binary (not counts)

# Add back age info
age_df = neds_diag[['key_ed', 'age']].drop_duplicates()
binary_matrix = binary_matrix.merge(age_df, left_index=True, right_on='key_ed')

# Save for clustering step
binary_matrix.to_csv("binary_comorbidity_matrix_2022.csv", index=False)

print("✅ Binary matrix created. Ready for clustering.")


In [ ]:
!pip install umap-learn hdbscan matplotlib seaborn --quiet

import pandas as pd
import umap
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns

# === Load binary matrix ===
df = pd.read_csv("binary_comorbidity_matrix_2022.csv")

# Define age groups

age_bins = [(60, 64), (65, 74), (75, 84), (85, 120)]
age_labels = ['60-64', '65-74', '75-84', '85+']

for (low, high), label in zip(age_bins, age_labels):
    print(f"\n🔍 Processing Age Group: {label}")

    # Subset by age group
    df_age = df[(df['age'] >= low) & (df['age'] <= high)].copy()
    X = df_age.drop(columns=['age', 'key_ed'], errors='ignore')

    # Dimensionality reduction with UMAP
    reducer = umap.UMAP(random_state=42)
    embedding = reducer.fit_transform(X)

    # Clustering with HDBSCAN
    clusterer = hdbscan.HDBSCAN(min_cluster_size=50, metric='euclidean')
    cluster_labels = clusterer.fit_predict(embedding)

    # Add cluster results
    df_age['cluster'] = cluster_labels
    df_age['umap_x'] = embedding[:, 0]
    df_age['umap_y'] = embedding[:, 1]

    # === Visualization ===
    plt.figure(figsize=(8, 6))
    sns.scatterplot(
        x='umap_x', y='umap_y', hue='cluster',
        data=df_age, palette='tab10', s=10
    )
    plt.title(f'UMAP + HDBSCAN Clusters for Age Group {label}, With AD')
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # === Cluster Profiles ===
    profile = df_age.groupby('cluster')[X.columns].mean().T
    profile = (profile * 100).round(1)  # prevalence in %
    print(f"\n📊 Cluster Comorbidity Profiles for {label}:")
    print(profile.head(10))  # show first 10 comorbidities per cluster


In [ ]:
# ===============================================================
#  Top-30 ICD-10 Comorbidities  |  NEDS 2022  (AD-only sample)
#  Upload together:
#     • NEDS_2022_Core_WORK_Age60-64-ALZ_ONLY.dta
#     • section111validicd10-jan2025_0.xlsx   (ICD-10 descriptions)
# ===============================================================
!pip install -q pyreadstat openpyxl

from google.colab import files
import pandas as pd, re, pyreadstat, pathlib

# ── 1 ▸ Upload the .dta and Section 111 Excel ──────────────────
uploaded  = files.upload()
dta_path  = next(p for p in uploaded if p.lower().endswith(".dta"))
xls_path  = next(p for p in uploaded if p != dta_path)

print("✔ NEDS file:", dta_path)
print("✔ ICD-10 lookup:", xls_path)

# ── 2 ▸ Read the NEDS file ─────────────────────────────────────
df, _ = pyreadstat.read_dta(dta_path, apply_value_formats=False)
print("Rows read:", len(df))

# ── 3 ▸ Find i10_dx1 … i10_dx35 columns (case-insensitive) ─────
dx_cols = sorted(
    [c for c in df.columns if re.fullmatch(r'i10_dx\d+', c, flags=re.IGNORECASE)],
    key=lambda x: int(re.search(r'\d+', x).group()))
if not dx_cols:
    raise RuntimeError("No i10_dx# columns found – check the dataset!")
print("Diagnosis columns:", dx_cols[:5], "… total", len(dx_cols))

# ── 4 ▸ Code cleaner for ICD-10 ────────────────────────────────
def clean(code):
    if pd.isna(code):           # true NaN
        return None
    c = str(code).strip().replace('.', '').upper()
    if c in {"", "0", "0.0"}:   # blank placeholders
        return None
    return c

# ── 5 ▸ Build ICD-10 → Description lookup from Section 111 ────
def load_lookup(path, use_long=False):
    tab = pd.read_excel(path, dtype=str)
    short = next(c for c in tab.columns if c.lower().startswith("short"))
    long  = next(c for c in tab.columns if c.lower().startswith("long"))
    col   = long if use_long else short
    return (tab[["CODE", col]]
              .rename(columns={"CODE":"ICD", col:"Description"})
              .assign(ICD=lambda d: d["ICD"].str.strip().str.upper())
              .drop_duplicates("ICD"))
lookup = load_lookup(xls_path, use_long=False)

# ── 6 ▸ Collapse diagnoses → long Series ───────────────────────
flat = (df[dx_cols].stack(dropna=True).map(clean).dropna())

# ── 7 ▸ Remove Alzheimer’s code itself (G30-series) ────────────
flat = flat[~flat.str.startswith("G30")]

# (optional) remove secondary‐dementia ICD-10 codes F02.8*, F02.9*
flat = flat[~flat.str.startswith(("F028", "F029"))]

# ── 8 ▸ Frequency table → top 30 ───────────────────────────────
top30 = (flat.value_counts().head(30)
         .rename_axis("ICD")
         .reset_index(name="Count"))

# ── 9 ▸ Merge in descriptions ─────────────────────────────────
top30 = (top30.merge(lookup, on="ICD", how="left")
               [["ICD", "Description", "Count"]])

# ── 10 ▸ % of Alzheimer ED discharges in this file ─────────────
top30["Pct_of_Discharges"] = (top30["Count"] / len(df) * 100).round(1)

# ── 11 ▸ Save & display ────────────────────────────────────────
top30 = top30.sort_values("Count", ascending=False).reset_index(drop=True)
top30.to_csv("NEDS_2022_Age60-64-AD_top30_comorbidities.csv", index=False)

pd.set_option("display.max_rows", None)
display(top30)
print("\n✅  Saved → NEDS_2022_Age60-64-AD_top30_comorbidities.csv")


In [ ]:
# ===============================================================
#  Top-30 ICD-10 Comorbidities  |  NEDS 2022  (AD-only sample)
#  Upload together:
#     • NEDS_2022_Core_WORK_Age65-74-ALZ_ONLY.dta
#     • section111validicd10-jan2025_0.xlsx   (ICD-10 descriptions)
# ===============================================================
!pip install -q pyreadstat openpyxl

from google.colab import files
import pandas as pd, re, pyreadstat, pathlib

# ── 1 ▸ Upload the .dta and Section 111 Excel ──────────────────
uploaded  = files.upload()
dta_path  = next(p for p in uploaded if p.lower().endswith(".dta"))
xls_path  = next(p for p in uploaded if p != dta_path)

print("✔ NEDS file:", dta_path)
print("✔ ICD-10 lookup:", xls_path)

# ── 2 ▸ Read the NEDS file ─────────────────────────────────────
df, _ = pyreadstat.read_dta(dta_path, apply_value_formats=False)
print("Rows read:", len(df))

# ── 3 ▸ Find i10_dx1 … i10_dx35 columns (case-insensitive) ─────
dx_cols = sorted(
    [c for c in df.columns if re.fullmatch(r'i10_dx\d+', c, flags=re.IGNORECASE)],
    key=lambda x: int(re.search(r'\d+', x).group()))
if not dx_cols:
    raise RuntimeError("No i10_dx# columns found – check the dataset!")
print("Diagnosis columns:", dx_cols[:5], "… total", len(dx_cols))

# ── 4 ▸ Code cleaner for ICD-10 ────────────────────────────────
def clean(code):
    if pd.isna(code):           # true NaN
        return None
    c = str(code).strip().replace('.', '').upper()
    if c in {"", "0", "0.0"}:   # blank placeholders
        return None
    return c

# ── 5 ▸ Build ICD-10 → Description lookup from Section 111 ────
def load_lookup(path, use_long=False):
    tab = pd.read_excel(path, dtype=str)
    short = next(c for c in tab.columns if c.lower().startswith("short"))
    long  = next(c for c in tab.columns if c.lower().startswith("long"))
    col   = long if use_long else short
    return (tab[["CODE", col]]
              .rename(columns={"CODE":"ICD", col:"Description"})
              .assign(ICD=lambda d: d["ICD"].str.strip().str.upper())
              .drop_duplicates("ICD"))
lookup = load_lookup(xls_path, use_long=False)

# ── 6 ▸ Collapse diagnoses → long Series ───────────────────────
flat = (df[dx_cols].stack(dropna=True).map(clean).dropna())

# ── 7 ▸ Remove Alzheimer’s code itself (G30-series) ────────────
flat = flat[~flat.str.startswith("G30")]

# (optional) remove secondary‐dementia ICD-10 codes F02.8*, F02.9*
flat = flat[~flat.str.startswith(("F028", "F029"))]

# ── 8 ▸ Frequency table → top 30 ───────────────────────────────
top30 = (flat.value_counts().head(30)
         .rename_axis("ICD")
         .reset_index(name="Count"))

# ── 9 ▸ Merge in descriptions ─────────────────────────────────
top30 = (top30.merge(lookup, on="ICD", how="left")
               [["ICD", "Description", "Count"]])

# ── 10 ▸ % of Alzheimer ED discharges in this file ─────────────
top30["Pct_of_Discharges"] = (top30["Count"] / len(df) * 100).round(1)

# ── 11 ▸ Save & display ────────────────────────────────────────
top30 = top30.sort_values("Count", ascending=False).reset_index(drop=True)
top30.to_csv("NEDS_2022_Age65-74-AD_top30_comorbidities.csv", index=False)

pd.set_option("display.max_rows", None)
display(top30)
print("\n✅  Saved → NEDS_2022_Age65-74-AD_top30_comorbidities.csv")


In [ ]:
# ===============================================================
#  Top-30 ICD-10 Comorbidities  |  NEDS 2022  (AD-only sample)
#  Upload together:
#     • NEDS_2022_Core_WORK_Age75-84-ALZ_ONLY.dta
#     • section111validicd10-jan2025_0.xlsx   (ICD-10 descriptions)
# ===============================================================
!pip install -q pyreadstat openpyxl

from google.colab import files
import pandas as pd, re, pyreadstat, pathlib

# ── 1 ▸ Upload the .dta and Section 111 Excel ──────────────────
uploaded  = files.upload()
dta_path  = next(p for p in uploaded if p.lower().endswith(".dta"))
xls_path  = next(p for p in uploaded if p != dta_path)

print("✔ NEDS file:", dta_path)
print("✔ ICD-10 lookup:", xls_path)

# ── 2 ▸ Read the NEDS file ─────────────────────────────────────
df, _ = pyreadstat.read_dta(dta_path, apply_value_formats=False)
print("Rows read:", len(df))

# ── 3 ▸ Find i10_dx1 … i10_dx35 columns (case-insensitive) ─────
dx_cols = sorted(
    [c for c in df.columns if re.fullmatch(r'i10_dx\d+', c, flags=re.IGNORECASE)],
    key=lambda x: int(re.search(r'\d+', x).group()))
if not dx_cols:
    raise RuntimeError("No i10_dx# columns found – check the dataset!")
print("Diagnosis columns:", dx_cols[:5], "… total", len(dx_cols))

# ── 4 ▸ Code cleaner for ICD-10 ────────────────────────────────
def clean(code):
    if pd.isna(code):           # true NaN
        return None
    c = str(code).strip().replace('.', '').upper()
    if c in {"", "0", "0.0"}:   # blank placeholders
        return None
    return c

# ── 5 ▸ Build ICD-10 → Description lookup from Section 111 ────
def load_lookup(path, use_long=False):
    tab = pd.read_excel(path, dtype=str)
    short = next(c for c in tab.columns if c.lower().startswith("short"))
    long  = next(c for c in tab.columns if c.lower().startswith("long"))
    col   = long if use_long else short
    return (tab[["CODE", col]]
              .rename(columns={"CODE":"ICD", col:"Description"})
              .assign(ICD=lambda d: d["ICD"].str.strip().str.upper())
              .drop_duplicates("ICD"))
lookup = load_lookup(xls_path, use_long=False)

# ── 6 ▸ Collapse diagnoses → long Series ───────────────────────
flat = (df[dx_cols].stack(dropna=True).map(clean).dropna())

# ── 7 ▸ Remove Alzheimer’s code itself (G30-series) ────────────
flat = flat[~flat.str.startswith("G30")]

# (optional) remove secondary‐dementia ICD-10 codes F02.8*, F02.9*
flat = flat[~flat.str.startswith(("F028", "F029"))]

# ── 8 ▸ Frequency table → top 30 ───────────────────────────────
top30 = (flat.value_counts().head(30)
         .rename_axis("ICD")
         .reset_index(name="Count"))

# ── 9 ▸ Merge in descriptions ─────────────────────────────────
top30 = (top30.merge(lookup, on="ICD", how="left")
               [["ICD", "Description", "Count"]])

# ── 10 ▸ % of Alzheimer ED discharges in this file ─────────────
top30["Pct_of_Discharges"] = (top30["Count"] / len(df) * 100).round(1)

# ── 11 ▸ Save & display ────────────────────────────────────────
top30 = top30.sort_values("Count", ascending=False).reset_index(drop=True)
top30.to_csv("NEDS_2022_Age75-84-AD_top30_comorbidities.csv", index=False)

pd.set_option("display.max_rows", None)
display(top30)
print("\n✅  Saved → NEDS_2022_Age75-84-AD_top30_comorbidities.csv")


In [ ]:
# ===============================================================
#  Top-30 ICD-10 Comorbidities  |  NEDS 2022  (AD-only sample)
#  Upload together:
#     • NEDS_2022_Core_WORK_Age85+ -ALZ_ONLY.dta
#     • section111validicd10-jan2025_0.xlsx   (ICD-10 descriptions)
# ===============================================================
!pip install -q pyreadstat openpyxl

from google.colab import files
import pandas as pd, re, pyreadstat, pathlib

# ── 1 ▸ Upload the .dta and Section 111 Excel ──────────────────
uploaded  = files.upload()
dta_path  = next(p for p in uploaded if p.lower().endswith(".dta"))
xls_path  = next(p for p in uploaded if p != dta_path)

print("✔ NEDS file:", dta_path)
print("✔ ICD-10 lookup:", xls_path)

# ── 2 ▸ Read the NEDS file ─────────────────────────────────────
df, _ = pyreadstat.read_dta(dta_path, apply_value_formats=False)
print("Rows read:", len(df))

# ── 3 ▸ Find i10_dx1 … i10_dx35 columns (case-insensitive) ─────
dx_cols = sorted(
    [c for c in df.columns if re.fullmatch(r'i10_dx\d+', c, flags=re.IGNORECASE)],
    key=lambda x: int(re.search(r'\d+', x).group()))
if not dx_cols:
    raise RuntimeError("No i10_dx# columns found – check the dataset!")
print("Diagnosis columns:", dx_cols[:5], "… total", len(dx_cols))

# ── 4 ▸ Code cleaner for ICD-10 ────────────────────────────────
def clean(code):
    if pd.isna(code):           # true NaN
        return None
    c = str(code).strip().replace('.', '').upper()
    if c in {"", "0", "0.0"}:   # blank placeholders
        return None
    return c

# ── 5 ▸ Build ICD-10 → Description lookup from Section 111 ────
def load_lookup(path, use_long=False):
    tab = pd.read_excel(path, dtype=str)
    short = next(c for c in tab.columns if c.lower().startswith("short"))
    long  = next(c for c in tab.columns if c.lower().startswith("long"))
    col   = long if use_long else short
    return (tab[["CODE", col]]
              .rename(columns={"CODE":"ICD", col:"Description"})
              .assign(ICD=lambda d: d["ICD"].str.strip().str.upper())
              .drop_duplicates("ICD"))
lookup = load_lookup(xls_path, use_long=False)

# ── 6 ▸ Collapse diagnoses → long Series ───────────────────────
flat = (df[dx_cols].stack(dropna=True).map(clean).dropna())

# ── 7 ▸ Remove Alzheimer’s code itself (G30-series) ────────────
flat = flat[~flat.str.startswith("G30")]

# (optional) remove secondary‐dementia ICD-10 codes F02.8*, F02.9*
flat = flat[~flat.str.startswith(("F028", "F029"))]

# ── 8 ▸ Frequency table → top 30 ───────────────────────────────
top30 = (flat.value_counts().head(30)
         .rename_axis("ICD")
         .reset_index(name="Count"))

# ── 9 ▸ Merge in descriptions ─────────────────────────────────
top30 = (top30.merge(lookup, on="ICD", how="left")
               [["ICD", "Description", "Count"]])

# ── 10 ▸ % of Alzheimer ED discharges in this file ─────────────
top30["Pct_of_Discharges"] = (top30["Count"] / len(df) * 100).round(1)

# ── 11 ▸ Save & display ────────────────────────────────────────
top30 = top30.sort_values("Count", ascending=False).reset_index(drop=True)
top30.to_csv("NEDS_2022_Age85plus-AD_top30_comorbidities.csv", index=False)

pd.set_option("display.max_rows", None)
display(top30)
print("\n✅  Saved → NEDS_2022_Age85plus-AD_top30_comorbidities.csv")


In [ ]:
# ╔════════════════════════════════════════════════════════╗
# ║     Comorbidity Clustering Pipeline for NEDS 2022 (ICD-10)     ║
# ╚════════════════════════════════════════════════════════╝

# 1. Install Required Libraries
!pip install pyreadstat mlxtend --quiet

# 2. Import Libraries
import pandas as pd
import numpy as np
import pyreadstat
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# 3. Load NEDS 2017 Alzheimer’s Sample (.dta format)
df, meta = pyreadstat.read_dta("/content/NEDS_2022_Core Alzheimers only age_60-64.dta")
print(f"Loaded data: {df.shape[0]} patients, {df.shape[1]} columns")

# 4. Extract and Clean ICD-10 Diagnosis Codes
dx_cols = sorted([col for col in df.columns if re.match(r'i10_dx\d+', col, re.IGNORECASE)])
def clean(code):
    if pd.isna(code): return None
    return str(code).strip().replace('.', '').upper()
for col in dx_cols:
    df[col] = df[col].apply(clean)

# 5. Define Top 30 ICD-10 Codes for Alzheimer’s Comorbidities
top_icd10 = [
    'Z20822', 'I10', 'Z79899', 'E785', 'K219', 'Z87891', 'E039', 'Z7982',
    'N179', 'Z66', 'E119', 'N390', 'G40909', 'F419', 'F32A', 'I2510',
    'Z8673', 'E876', 'G9341', 'E860', 'J449', 'F17210', 'Z794', 'D649',
    'U071', 'Z7984', 'Z7901', 'J9601', 'A419', 'E1122'
]

# 6. Create Binary Comorbidity Matrix
def has_code(row, code):
    return int(code in row.values)
binary_matrix = pd.DataFrame(0, index=df.index, columns=top_icd10)
for code in top_icd10:
    binary_matrix[code] = df[dx_cols].apply(lambda row: has_code(row, code), axis=1)

# 7. Standardize and Run KMeans Clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(binary_matrix)

# Optional: Elbow Plot to Choose K
inertia = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)
plt.plot(range(2, 10), inertia, marker='o')
plt.title("Elbow Method to Choose k")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

# 8. Fit KMeans with Optimal k (Adjust as Needed)
kmeans = KMeans(n_clusters=8, n_init=10, random_state=42)
df['cluster_kmeans'] = kmeans.fit_predict(X_scaled)

# 9. Profile Clusters
cluster_profiles = binary_matrix.copy()
cluster_profiles['cluster'] = df['cluster_kmeans']
summary = cluster_profiles.groupby('cluster').mean().T.round(2)

# ✅ ICD-10 → Comorbidity Name Mapping

# ✅ ICD-10 → Comorbidity Name Mapping (Updated)
icd10_to_name = {
    'Z20822': 'Contact with and (suspected) exposure to COVID-19',
    'I10': 'Essential (primary) hypertension',
    'Z79899': 'Other long-term (current) drug therapy',
    'E785': 'Hyperlipidemia, unspecified',
    'K219': 'Gastro-esophageal reflux disease without esophagitis',
    'Z87891': 'Personal history of nicotine dependence',
    'E039': 'Hypothyroidism, unspecified',
    'Z7982': 'Long-term (current) use of aspirin',
    'N179': 'Acute kidney failure, unspecified',
    'Z66': 'Do not resuscitate status',
    'E119': 'Type 2 diabetes mellitus without complications',
    'N390': 'Urinary tract infection, site not specified',
    'G40909': 'Epilepsy, not intractable, without status epilepticus',
    'F419': 'Anxiety disorder, unspecified',
    'F32A': 'Depression, unspecified',
    'I2510': 'Atherosclerotic heart disease of native coronary artery without angina pectoris',
    'Z8673': 'Personal history of transient ischemic attack (TIA)',
    'E876': 'Hypokalemia',
    'G9341': 'Metabolic encephalopathy',
    'E860': 'Dehydration',
    'J449': 'Chronic obstructive pulmonary disease, unspecified',
    'F17210': 'Nicotine dependence, cigarettes, uncomplicated',
    'Z794': 'Long-term (current) use of insulin',
    'D649': 'Anemia, unspecified',
    'U071': 'COVID-19, virus identified',
    'Z7984': 'Long-term (current) use of oral hypoglycemics',
    'Z7901': 'Long-term (current) use of anticoagulants',
    'J9601': 'Acute respiratory failure with hypoxia',
    'A419': 'Sepsis, unspecified organism',
    'E1122': 'Type 2 diabetes mellitus with diabetic chronic kidney disease'
}

summary.index = [icd10_to_name.get(code, code) for code in summary.index]

# 10. Heatmap of Comorbidity Presence by Cluster
plt.figure(figsize=(12, 10))
sns.heatmap(summary, cmap='Blues', annot=True, fmt=".2f")
plt.title("Diagnostic Code Clusters For Age Group 60-64, With AD")
plt.xlabel("Cluster")
plt.ylabel("Comorbidity")
plt.tight_layout()
plt.show()





In [ ]:
# ╔════════════════════════════════════════════════════════╗
# ║     Comorbidity Clustering Pipeline for NEDS 2022 (ICD-10)     ║
# ╚════════════════════════════════════════════════════════╝

# 1. Install Required Libraries
!pip install pyreadstat mlxtend --quiet

# 2. Import Libraries
import pandas as pd
import numpy as np
import pyreadstat
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# 3. Load NEDS 2017 Alzheimer’s Sample (.dta format)
df, meta = pyreadstat.read_dta("/content/NEDS_2022_Core Alzheimers only age_65-74.dta")
print(f"Loaded data: {df.shape[0]} patients, {df.shape[1]} columns")

# 4. Extract and Clean ICD-10 Diagnosis Codes
dx_cols = sorted([col for col in df.columns if re.match(r'i10_dx\d+', col, re.IGNORECASE)])
def clean(code):
    if pd.isna(code): return None
    return str(code).strip().replace('.', '').upper()
for col in dx_cols:
    df[col] = df[col].apply(clean)

# 5. Define Top 30 ICD-10 Codes for Alzheimer’s Comorbidities
top_icd10 = [
    'I10', 'Z20822', 'Z79899', 'E785', 'Z87891', 'Z7982', 'E119', 'I2510', 'K219',
    'Z7901', 'F17210', 'Z7984', 'J449', 'N179', 'E039', 'Z794', 'E1122', 'F419',
    'U071', 'I110', 'E876', 'N390', 'Z8673', 'E7800', 'E871', 'I4891', 'F32A',
    'E669', 'I252', 'I129'
]


# 6. Create Binary Comorbidity Matrix
def has_code(row, code):
    return int(code in row.values)
binary_matrix = pd.DataFrame(0, index=df.index, columns=top_icd10)
for code in top_icd10:
    binary_matrix[code] = df[dx_cols].apply(lambda row: has_code(row, code), axis=1)

# 7. Standardize and Run KMeans Clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(binary_matrix)

# Optional: Elbow Plot to Choose K
inertia = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)
plt.plot(range(2, 10), inertia, marker='o')
plt.title("Elbow Method to Choose k")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

# 8. Fit KMeans with Optimal k (Adjust as Needed)
kmeans = KMeans(n_clusters=8, n_init=10, random_state=42)
df['cluster_kmeans'] = kmeans.fit_predict(X_scaled)

# 9. Profile Clusters
cluster_profiles = binary_matrix.copy()
cluster_profiles['cluster'] = df['cluster_kmeans']
summary = cluster_profiles.groupby('cluster').mean().T.round(2)

# ✅ ICD-10 → Comorbidity Name Mapping

# ✅ ICD-10 → Comorbidity Name Mapping (Updated)
icd10_to_name = {
    'I10': 'Essential (primary) hypertension',
    'Z20822': 'Contact with and (suspected) exposure to COVID-19',
    'Z79899': 'Other long-term (current) drug therapy',
    'E785': 'Hyperlipidemia, unspecified',
    'Z87891': 'Personal history of nicotine dependence',
    'Z7982': 'Long-term (current) use of aspirin',
    'E119': 'Type 2 diabetes mellitus without complications',
    'I2510': 'Atherosclerotic heart disease of native coronary artery without angina pectoris',
    'K219': 'Gastro-esophageal reflux disease without esophagitis',
    'Z7901': 'Long-term (current) use of anticoagulants',
    'F17210': 'Nicotine dependence, cigarettes, uncomplicated',
    'Z7984': 'Long-term (current) use of oral hypoglycemic drugs',
    'J449': 'Chronic obstructive pulmonary disease, unspecified',
    'N179': 'Acute kidney failure, unspecified',
    'E039': 'Hypothyroidism, unspecified',
    'Z794': 'Long-term (current) use of insulin',
    'E1122': 'Type 2 diabetes mellitus with diabetic chronic kidney disease',
    'F419': 'Anxiety disorder, unspecified',
    'U071': 'COVID-19, virus identified',
    'I110': 'Hypertensive heart disease with heart failure',
    'E876': 'Hypokalemia',
    'N390': 'Urinary tract infection, site not specified',
    'Z8673': 'Personal history of transient ischemic attack (TIA)',
    'E7800': 'Pure hypercholesterolemia, unspecified',
    'E871': 'Hypo-osmolality and hyponatremia',
    'I4891': 'Unspecified atrial fibrillation',
    'F32A': 'Depression, unspecified',
    'E669': 'Obesity, unspecified',
    'I252': 'Old myocardial infarction',
    'I129': 'Hypertensive chronic kidney disease with stage 1 through stage 4'
}


summary.index = [icd10_to_name.get(code, code) for code in summary.index]

# 10. Heatmap of Comorbidity Presence by Cluster
plt.figure(figsize=(12, 10))
sns.heatmap(summary, cmap='Blues', annot=True, fmt=".2f")
plt.title("Diagnostic Code Clusters For Age Group 65-74, With AD")
plt.xlabel("Cluster")
plt.ylabel("Comorbidity")
plt.tight_layout()
plt.show()





In [ ]:
# ╔════════════════════════════════════════════════════════╗
# ║     Comorbidity Clustering Pipeline for NEDS 2022 (ICD-10)     ║
# ╚════════════════════════════════════════════════════════╝

# 1. Install Required Libraries
!pip install pyreadstat mlxtend --quiet

# 2. Import Libraries
import pandas as pd
import numpy as np
import pyreadstat
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# 3. Load NEDS 2017 Alzheimer’s Sample (.dta format)
df, meta = pyreadstat.read_dta("/content/NEDS_2022_Core Alzheimers only age_75-84.dta")
print(f"Loaded data: {df.shape[0]} patients, {df.shape[1]} columns")

# 4. Extract and Clean ICD-10 Diagnosis Codes
dx_cols = sorted([col for col in df.columns if re.match(r'i10_dx\d+', col, re.IGNORECASE)])
def clean(code):
    if pd.isna(code): return None
    return str(code).strip().replace('.', '').upper()
for col in dx_cols:
    df[col] = df[col].apply(clean)

# 5. Define Top 30 ICD-10 Codes for Alzheimer’s Comorbidities
top_icd10 = [
    'I10', 'Z20822', 'E785', 'Z79899', 'Z66', 'Z87891', 'I2510', 'Z7982', 'K219',
    'N179', 'N390', 'E039', 'Z7901', 'G9341', 'E860', 'E119', 'Z8673', 'E876',
    'I129', 'F32A', 'E1122', 'U071', 'F419', 'Z7984', 'J449', 'Z515', 'J9601',
    'I4891', 'D649', 'I110'
]

# 6. Create Binary Comorbidity Matrix
def has_code(row, code):
    return int(code in row.values)
binary_matrix = pd.DataFrame(0, index=df.index, columns=top_icd10)
for code in top_icd10:
    binary_matrix[code] = df[dx_cols].apply(lambda row: has_code(row, code), axis=1)

# 7. Standardize and Run KMeans Clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(binary_matrix)

# Optional: Elbow Plot to Choose K
inertia = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)
plt.plot(range(2, 10), inertia, marker='o')
plt.title("Elbow Method to Choose k")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

# 8. Fit KMeans with Optimal k (Adjust as Needed)
kmeans = KMeans(n_clusters=8, n_init=10, random_state=42)
df['cluster_kmeans'] = kmeans.fit_predict(X_scaled)

# 9. Profile Clusters
cluster_profiles = binary_matrix.copy()
cluster_profiles['cluster'] = df['cluster_kmeans']
summary = cluster_profiles.groupby('cluster').mean().T.round(2)

# ✅ ICD-10 → Comorbidity Name Mapping

# ✅ ICD-10 → Comorbidity Name Mapping (Updated)
icd10_to_name = {
    'I10': 'Essential (primary) hypertension',
    'Z20822': 'Contact with and (suspected) exposure to COVID-19',
    'E785': 'Hyperlipidemia, unspecified',
    'Z79899': 'Other long-term (current) drug therapy',
    'Z66': 'Do Not Resuscitate status',
    'Z87891': 'Personal history of nicotine dependence',
    'I2510': 'Atherosclerotic heart disease of native coronary artery without angina pectoris',
    'Z7982': 'Long-term (current) use of aspirin',
    'K219': 'Gastro-esophageal reflux disease without esophagitis',
    'N179': 'Acute kidney failure, unspecified',
    'N390': 'Urinary tract infection, site not specified',
    'E039': 'Hypothyroidism, unspecified',
    'Z7901': 'Long-term (current) use of anticoagulants',
    'G9341': 'Metabolic encephalopathy',
    'E860': 'Dehydration',
    'E119': 'Type 2 diabetes mellitus without complications',
    'Z8673': 'Personal history of transient ischemic attack (TIA)',
    'E876': 'Hypokalemia',
    'I129': 'Hypertensive chronic kidney disease with stage 1–4',
    'F32A': 'Depression, unspecified',
    'E1122': 'Type 2 diabetes mellitus with diabetic chronic kidney disease',
    'U071': 'COVID-19, virus identified',
    'F419': 'Anxiety disorder, unspecified',
    'Z7984': 'Long-term (current) use of oral hypoglycemics',
    'J449': 'Chronic obstructive pulmonary disease, unspecified',
    'Z515': 'Encounter for palliative care',
    'J9601': 'Acute respiratory failure with hypoxia',
    'I4891': 'Unspecified atrial fibrillation',
    'D649': 'Anemia, unspecified',
    'I110': 'Hypertensive heart disease with heart failure'
}


summary.index = [icd10_to_name.get(code, code) for code in summary.index]

# 10. Heatmap of Comorbidity Presence by Cluster
plt.figure(figsize=(12, 10))
sns.heatmap(summary, cmap='Blues', annot=True, fmt=".2f")
plt.title("Diagnostic Code Clusters For Age Group 75-84, With AD")
plt.xlabel("Cluster")
plt.ylabel("Comorbidity")
plt.tight_layout()
plt.show()



In [ ]:
# ╔════════════════════════════════════════════════════════╗
# ║     Comorbidity Clustering Pipeline for NEDS 2022 (ICD-10)     ║
# ╚════════════════════════════════════════════════════════╝

# 1. Install Required Libraries
!pip install pyreadstat mlxtend --quiet

# 2. Import Libraries
import pandas as pd
import numpy as np
import pyreadstat
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# 3. Load NEDS 2017 Alzheimer’s Sample (.dta format)
df, meta = pyreadstat.read_dta("/content/NEDS_2022_Core Alzheimers only age_85+.dta")
print(f"Loaded data: {df.shape[0]} patients, {df.shape[1]} columns")

# 4. Extract and Clean ICD-10 Diagnosis Codes
dx_cols = sorted([col for col in df.columns if re.match(r'i10_dx\d+', col, re.IGNORECASE)])
def clean(code):
    if pd.isna(code): return None
    return str(code).strip().replace('.', '').upper()
for col in dx_cols:
    df[col] = df[col].apply(clean)

# 5. Define Top 30 ICD-10 Codes for Alzheimer’s Comorbidities
top_icd10 = [
    'Z20822', 'I10', 'E785', 'Z66', 'Z79899', 'I2510', 'N179', 'E039', 'Z87891',
    'N390', 'Z7982', 'K219', 'Z7901', 'E860', 'I129', 'G9341', 'Z515', 'Z8673',
    'I4891', 'E876', 'U071', 'I110', 'J9601', 'E119', 'N1830', 'F32A', 'E1122',
    'D649', 'I480', 'I130'
]


# 6. Create Binary Comorbidity Matrix
def has_code(row, code):
    return int(code in row.values)
binary_matrix = pd.DataFrame(0, index=df.index, columns=top_icd10)
for code in top_icd10:
    binary_matrix[code] = df[dx_cols].apply(lambda row: has_code(row, code), axis=1)

# 7. Standardize and Run KMeans Clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(binary_matrix)

# Optional: Elbow Plot to Choose K
inertia = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)
plt.plot(range(2, 10), inertia, marker='o')
plt.title("Elbow Method to Choose k")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

# 8. Fit KMeans with Optimal k (Adjust as Needed)
kmeans = KMeans(n_clusters=8, n_init=10, random_state=42)
df['cluster_kmeans'] = kmeans.fit_predict(X_scaled)

# 9. Profile Clusters
cluster_profiles = binary_matrix.copy()
cluster_profiles['cluster'] = df['cluster_kmeans']
summary = cluster_profiles.groupby('cluster').mean().T.round(2)

# ✅ ICD-10 → Comorbidity Name Mapping

icd10_to_name = {
    'Z20822': 'Contact with and (suspected) exposure to COVID-19',
    'I10': 'Essential (primary) hypertension',
    'E785': 'Hyperlipidemia, unspecified',
    'Z66': 'Do not resuscitate status',
    'Z79899': 'Other long-term (current) drug therapy',
    'I2510': 'Atherosclerotic heart disease of native coronary artery w/o angina',
    'N179': 'Acute kidney failure, unspecified',
    'E039': 'Hypothyroidism, unspecified',
    'Z87891': 'Personal history of nicotine dependence',
    'N390': 'Urinary tract infection, site not specified',
    'Z7982': 'Long-term (current) use of aspirin',
    'K219': 'Gastro-esophageal reflux disease without esophagitis',
    'Z7901': 'Long-term (current) use of anticoagulants',
    'E860': 'Dehydration',
    'I129': 'Hypertensive chronic kidney disease stage 1-4',
    'G9341': 'Metabolic encephalopathy',
    'Z515': 'Encounter for palliative care',
    'Z8673': 'Personal history of transient ischemic attack (TIA)',
    'I4891': 'Unspecified atrial fibrillation',
    'E876': 'Hypokalemia',
    'U071': 'COVID-19',
    'I110': 'Hypertensive heart disease with heart failure',
    'J9601': 'Acute respiratory failure with hypoxia',
    'E119': 'Type 2 diabetes mellitus without complications',
    'N1830': 'Chronic kidney disease, stage 3, unspecified',
    'F32A': 'Depression, unspecified',
    'E1122': 'Type 2 diabetes mellitus with diabetic chronic kidney disease',
    'D649': 'Anemia, unspecified',
    'I480': 'Paroxysmal atrial fibrillation',
    'I130': 'Hypertensive heart and chronic kidney disease with heart failure, stage 1–4'
}


summary.index = [icd10_to_name.get(code, code) for code in summary.index]

# 10. Heatmap of Comorbidity Presence by Cluster
plt.figure(figsize=(12, 10))
sns.heatmap(summary, cmap='Blues', annot=True, fmt=".2f")
plt.title("Diagnostic Code Clusters For Age Group 85+, With AD")
plt.xlabel("Cluster")
plt.ylabel("Comorbidity")
plt.tight_layout()
plt.show()


